# NYC Congestion Pricing Analysis

This notebook downloads and analyzes NYC TLC (Taxi & Limousine Commission) trip record data.

## Data Source

NYC publishes taxi trip data at: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

The data includes:
- **Yellow Taxi**: Traditional yellow cabs
- **Green Taxi**: Street hail livery vehicles
- **For-Hire Vehicle (FHV)**: Standard FHV services
- **High Volume FHV (HVFHV)**: Rideshare services

All files are in Parquet format with the naming pattern:
`https://d37ci6vzurychx.cloudfront.net/trip-data/{type}_tripdata_{year}-{month}.parquet`

## Example Prompt for Downloading Data

To download the data, you can use this prompt with Claude Code in the project directory:

```
Download the last 5 years of NYC Yellow Taxi trip data in Parquet format 
from the NYC TLC website. Use Playwright to parse the page and get the 
most recent available data files. Save them to the data/ directory.
```

In [ ]:
# Parameters (tagged for papermill)
data_type = "yellow"  # Options: yellow, green, fhv, fhvhv
years_back = 5  # Number of years of historical data to download
data_dir = "data"
output_path = "outputs/result.parquet"
use_playwright = False  # Set to True to dynamically parse the TLC website

In [2]:
import duckdb
import polars as pl
from pathlib import Path
from datetime import datetime, timedelta
from urllib.request import urlretrieve
import calendar

In [3]:
# Create data directory if it doesn't exist
Path(data_dir).mkdir(parents=True, exist_ok=True)
print(f"Data directory: {Path(data_dir).absolute()}")

Data directory: c:\Users\Rob\Desktop\ai-for-the-rest\examples\nyc-congestion-pricing\notebooks\data


## Download NYC TLC Data

Two approaches are provided:

1. **Direct Download** (default): Uses the known URL pattern to download files
2. **Playwright-based** (optional): Dynamically parses the TLC website to get the latest available files

The direct approach is faster and doesn't require browser automation. Use Playwright if you need to ensure you're getting the absolute latest data or if the URL pattern changes.

In [4]:
def download_tlc_data_direct(data_type, years_back, data_dir):
    """
    Download NYC TLC data using direct URL pattern.
    
    Args:
        data_type: Type of data (yellow, green, fhv, fhvhv)
        years_back: Number of years of historical data
        data_dir: Directory to save files
    
    Returns:
        List of downloaded file paths
    """
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data"
    downloaded_files = []
    
    # Calculate date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365 * years_back)
    
    # Generate list of year-month combinations
    current = start_date.replace(day=1)
    while current <= end_date:
        year = current.year
        month = current.month
        
        # Construct filename and URL
        filename = f"{data_type}_tripdata_{year}-{month:02d}.parquet"
        url = f"{base_url}/{filename}"
        local_path = Path(data_dir) / filename
        
        # Download if not already present
        if not local_path.exists():
            try:
                print(f"Downloading {filename}...")
                urlretrieve(url, local_path)
                downloaded_files.append(str(local_path))
                print(f"  ✓ Saved to {local_path}")
            except Exception as e:
                print(f"  ✗ Failed to download {filename}: {e}")
        else:
            print(f"  ⊙ {filename} already exists, skipping")
            downloaded_files.append(str(local_path))
        
        # Move to next month
        if month == 12:
            current = current.replace(year=year + 1, month=1)
        else:
            current = current.replace(month=month + 1)
    
    return downloaded_files

print("Direct download function defined")

Direct download function defined


In [5]:
def download_tlc_data_playwright(data_type, years_back, data_dir):
    """
    Download NYC TLC data by parsing the official website with Playwright.
    This ensures you get the latest available data even if URL patterns change.
    
    Requires: playwright (install with: playwright install)
    
    Args:
        data_type: Type of data (yellow, green, fhv, fhvhv)
        years_back: Number of years of historical data
        data_dir: Directory to save files
    
    Returns:
        List of downloaded file paths
    """
    try:
        from playwright.sync_api import sync_playwright
    except ImportError:
        print("Error: Playwright not installed. Install with: pip install playwright && playwright install")
        return []
    
    downloaded_files = []
    
    # Calculate cutoff date
    cutoff_date = datetime.now() - timedelta(days=365 * years_back)
    
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        
        print("Loading NYC TLC trip record data page...")
        page.goto("https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page")
        page.wait_for_load_state("networkidle")
        
        # Find all parquet download links
        print(f"Parsing page for {data_type} parquet files...")
        links = page.query_selector_all("a[href*='.parquet']")
        
        for link in links:
            href = link.get_attribute("href")
            if not href or data_type not in href:
                continue
            
            # Extract year and month from filename
            # Expected format: {type}_tripdata_{year}-{month}.parquet
            try:
                parts = href.split("/")[-1].replace(".parquet", "").split("_")
                if len(parts) >= 3:
                    year_month = parts[2]  # e.g., "2025-01"
                    year, month = map(int, year_month.split("-"))
                    file_date = datetime(year, month, 1)
                    
                    # Skip if before cutoff date
                    if file_date < cutoff_date:
                        continue
                    
                    filename = href.split("/")[-1]
                    local_path = Path(data_dir) / filename
                    
                    # Download if not already present
                    if not local_path.exists():
                        print(f"Downloading {filename}...")
                        full_url = href if href.startswith("http") else f"https://d37ci6vzurychx.cloudfront.net{href}"
                        urlretrieve(full_url, local_path)
                        downloaded_files.append(str(local_path))
                        print(f"  ✓ Saved to {local_path}")
                    else:
                        print(f"  ⊙ {filename} already exists, skipping")
                        downloaded_files.append(str(local_path))
            except Exception as e:
                print(f"  ✗ Error processing {href}: {e}")
                continue
        
        browser.close()
    
    return downloaded_files

print("Playwright download function defined")

Playwright download function defined


In [6]:
# Execute download
print(f"Starting download: {data_type} taxi data for the last {years_back} years")
print(f"Method: {'Playwright (dynamic parsing)' if use_playwright else 'Direct (URL pattern)'}")
print("-" * 60)

if use_playwright:
    data_files = download_tlc_data_playwright(data_type, years_back, data_dir)
else:
    data_files = download_tlc_data_direct(data_type, years_back, data_dir)

print("-" * 60)
print(f"Download complete! {len(data_files)} files available for analysis")

# Filter out files that don't exist (failed downloads)
data_files = [f for f in data_files if Path(f).exists()]
print(f"Successfully downloaded/found: {len(data_files)} files")

Starting download: yellow taxi data for the last 5 years
Method: Direct (URL pattern)
------------------------------------------------------------
  ✓ Saved to data\yellow_tripdata_2020-12.parquet
  ✓ Saved to data\yellow_tripdata_2021-01.parquet
  ✓ Saved to data\yellow_tripdata_2021-02.parquet
  ✓ Saved to data\yellow_tripdata_2021-03.parquet
  ✓ Saved to data\yellow_tripdata_2021-04.parquet
  ✓ Saved to data\yellow_tripdata_2021-05.parquet
  ✓ Saved to data\yellow_tripdata_2021-06.parquet
  ✓ Saved to data\yellow_tripdata_2021-07.parquet
  ✓ Saved to data\yellow_tripdata_2021-08.parquet
  ✓ Saved to data\yellow_tripdata_2021-09.parquet
  ✓ Saved to data\yellow_tripdata_2021-10.parquet
  ✓ Saved to data\yellow_tripdata_2021-11.parquet
  ✓ Saved to data\yellow_tripdata_2021-12.parquet
  ✓ Saved to data\yellow_tripdata_2022-01.parquet
  ✓ Saved to data\yellow_tripdata_2022-02.parquet
  ✓ Saved to data\yellow_tripdata_2022-03.parquet
  ✓ Saved to data\yellow_tripdata_2022-04.parquet
  ✓

## Data Analysis with DuckDB

Now that we have the data, let's analyze it using DuckDB. DuckDB can query Parquet files directly without loading them into memory, making it perfect for large datasets.

In [7]:
# Query data directly from Parquet files using DuckDB
# Use glob pattern to query all downloaded files at once
if data_files:
    pattern = f"{data_dir}/{data_type}_tripdata_*.parquet"
    
    # Get basic statistics
    query = f"""
    SELECT 
        COUNT(*) as total_trips,
        MIN(tpep_pickup_datetime) as earliest_trip,
        MAX(tpep_pickup_datetime) as latest_trip,
        AVG(trip_distance) as avg_distance,
        AVG(total_amount) as avg_fare
    FROM '{pattern}'
    """
    
    stats = duckdb.sql(query).pl()
    print("Dataset Statistics:")
    print(stats)
else:
    print("No data files available for analysis")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ModuleNotFoundError: No module named 'pyarrow'

In [ ]:
# Analyze trips by month and year - convert to Polars for further processing
if data_files:
    query = f"""
    SELECT 
        YEAR(tpep_pickup_datetime) as year,
        MONTH(tpep_pickup_datetime) as month,
        COUNT(*) as trip_count,
        AVG(trip_distance) as avg_distance,
        AVG(total_amount) as avg_fare,
        SUM(total_amount) as total_revenue
    FROM '{pattern}'
    GROUP BY year, month
    ORDER BY year, month
    """
    
    monthly_stats = duckdb.sql(query).pl()
    
    # Use Polars to add year-month string for better display
    monthly_stats = monthly_stats.with_columns(
        (pl.col("year").cast(str) + "-" + pl.col("month").cast(str).str.zfill(2)).alias("year_month")
    )
    
    print(f"\nMonthly Statistics ({len(monthly_stats)} months):")
    print(monthly_stats.head(12))

In [ ]:
# Analyze pickup locations (Location IDs for congestion pricing zones)
# Manhattan's congestion pricing zone is roughly LocationIDs in certain ranges
if data_files:
    query = f"""
    SELECT 
        PULocationID as pickup_location,
        COUNT(*) as trip_count,
        AVG(trip_distance) as avg_distance,
        AVG(total_amount) as avg_fare
    FROM '{pattern}'
    WHERE PULocationID IS NOT NULL
    GROUP BY PULocationID
    ORDER BY trip_count DESC
    LIMIT 20
    """
    
    top_locations = duckdb.sql(query).pl()
    
    print("\nTop 20 Pickup Locations by Trip Count:")
    print(top_locations)

In [ ]:
# Save aggregated results to outputs directory
Path("outputs").mkdir(parents=True, exist_ok=True)

if data_files and len(monthly_stats) > 0:
    # Save monthly statistics
    monthly_stats.write_parquet(output_path)
    print(f"\nResults saved to {output_path}")
    print(f"Total months analyzed: {len(monthly_stats)}")
    print(f"Date range: {monthly_stats['year_month'].min()} to {monthly_stats['year_month'].max()}")
else:
    print("\nNo results to save")